# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [40]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests


# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [7]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")

# Display sample data
vacation_df.head()

,Unnamed: 0,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,8,Atuona,PF,77.85,light rain,-9.8000,-139.0333,Pearl Resort
1,10,Vaini,TO,84.36,broken clouds,-21.2000,-175.2000,Keleti International Resort
2,11,Ballina,AU,71.02,clear sky,-28.8667,153.5667,Ballina Beach Resort
3,15,Puerto Ayora,EC,78.55,overcast clouds,-0.7393,-90.3518,Hostal La Mirada De Solitario George
4,21,Rikitea,PF,77.81,broken clouds,-23.1203,-134.9692,Chez Bianca & Benoit


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [9]:
# Configure the map plot
hotel_map = vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country", "Current Description"]
)

# Display the map plot
hotel_map

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Current Description)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [10]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method

vacation_start = vacation_df.loc[vacation_df["City"] == "Ballina"]
vacation_end = vacation_df.loc[vacation_df["City"] == "Ballina"]
vacation_stop1 = vacation_df.loc[vacation_df["City"] == "Yeppoon"]
vacation_stop2 = vacation_df.loc[vacation_df["City"] == "Mackay"]
vacation_stop3 = vacation_df.loc[vacation_df["City"] == "Bowen"]

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [11]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = pd.concat(
   [            
    vacation_start,
    vacation_stop1,
    vacation_stop2,
    vacation_stop3,
    vacation_end,
   ],
    ignore_index = True
)
                    

# Display sample data
itinerary_df

,Unnamed: 0,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,11,Ballina,AU,71.02,clear sky,-28.8667,153.5667,Ballina Beach Resort
1,283,Yeppoon,AU,73.18,broken clouds,-23.1333,150.7333,Blossom's on Seaspray
2,180,Mackay,AU,78.67,broken clouds,-21.1500,149.2000,Mackay Oceanside Central Apartment Hotel
3,473,Bowen,AU,75.31,scattered clouds,-20.0167,148.2333,Whitsunday Sands Resort
4,11,Ballina,AU,71.02,clear sky,-28.8667,153.5667,Ballina Beach Resort


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [12]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[["Lng", "Lat"]].copy()


# Display sample data
waypoints_df

,Lng,Lat
0,153.5667,-28.8667
1,150.7333,-23.1333
2,149.2000,-21.1500
3,148.2333,-20.0167
4,153.5667,-28.8667


## Use GeoViews to create map that shows the four cities in the itinerary

In [21]:
# Configure the map plot by using the itineraty_df
waypoints_map =  itinerary_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 1,
    size = "Max Temp",
    color = "City",
    hover_cols = ["Hotel Name", "Country", "Current Description"]
)

# Display the map plot
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Current Description)

In [ ]:
# Display the route_map
# YOUR CODE HERE

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [22]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [23]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': 'e6a9fecec04c4759ad36e6dc50b53170',
 'waypoints': '-28.8667,153.5667|-23.1333,150.7333|-21.15,149.2|-20.0167,148.2333|-28.8667,153.5667'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [24]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params)

# Convert the API response to JSON format
route_response = route_response.json()

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [28]:
# Fetch the route's legs coordinates from the JSON reponse
legs = route_response["features"][0]["geometry"]["coordinates"]


## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [29]:
# Create and empty list to store the longitude of each step
longitude = []

# Create and empty list to store the latitude of step
latitude = []

# Loop through the legs coordinates to fetch the latitude and longitude for each step
for leg in legs:
    for coordinate in leg:
        longitude.append(coordinate[0])
        latitude.append(coordinate[1])

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [30]:
# Create an empty DataFrame to store the steps' coordinates
route_df = pd.DataFrame()

# Add the steps' longitude and latitude from each step as columns to the DataFrame
route_df["longitude"] = longitude
route_df["latitude"] = latitude

# Display sample data
route_df.head()

,longitude,latitude
0,153.566938,-28.866806
1,153.567416,-28.865984
2,153.567259,-28.865914
3,153.566926,-28.865836
4,153.566902,-28.865846


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [31]:
# Configure the route path by using the GeoViews' Path function
route_path = gv.Path(
    route_df,
).opts(
    height=500,
    width=700,
    line_width = 3,
    color = "red"
)

In [36]:
# Display a composed plot by using the route_map and route_path objects
WeatherPy_travel_map = waypoints_map * route_path
WeatherPy_travel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Current Description)
   .Path.I   :Path   [longitude,latitude]